### @author: ODD team
# XGBoost model- this is adapted
  
#### You cau use this form about training.

In [ ]:
# Import Module
from tqdm import tqdm
import os
import pandas as pd
import numpy as np
import time
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn import ensemble
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from custom_datasets import CustomDataset
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from tqdm import tqdm
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import StratifiedKFold
import xgboost as xgb
import warnings
from sklearn.model_selection import KFold

In [ ]:
warnings.filterwarnings(action='ignore')
from sklearn.metrics import make_scorer
#define your own mse and set greater_is_better=False
mse = make_scorer(mean_squared_error,greater_is_better=False)

In [ ]:
# train valid test set
df_train = pd.read_csv('../datasets/kitti_train_1.csv')
df_valid = pd.read_csv('../datasets/kitti_valid_1.csv')
df_test = pd.read_csv('../datasets/kitti_test_1.csv')

In [ ]:
train = df_train.drop(['filename', 'weather','depth_min'], axis=1)
valid = df_valid.drop(['filename', 'weather','depth_min'], axis=1)
test = df_test.drop(['filename', 'weather','depth_min'], axis=1)

In [ ]:
# encoding
le=LabelEncoder()
tr=le.fit_transform(train['class'].values)
vv=le.fit_transform(valid['class'].values)
ts=le.fit_transform(test['class'].values)

train['class']=tr
valid['class']=vv
test['class']=ts

In [ ]:
train = train[train['zloc'] < 90].reset_index(drop=True)
valid = valid[valid['zloc'] < 90].reset_index(drop=True)
test = pd.DataFrame(test[test['zloc']< 90]).reset_index(drop=True)


X_train = train.loc[:, train.columns != 'zloc']
y_train = train.loc[:, train.columns == 'zloc']

X_valid = valid.loc[:, valid.columns != 'zloc']
y_valid = valid.loc[:, valid.columns == 'zloc']

X_test = test.loc[:, test.columns != 'zloc']
y_test = test.loc[:, test.columns == 'zloc']

In [ ]:
X_train.head()

In [ ]:
# scaling
from sklearn.preprocessing import StandardScaler
scalerX = StandardScaler().fit(X_train)
#scalery = StandardScaler().fit(y_train)

In [ ]:
X_train= scalerX.transform(X_train)
#y_train_scale = scalery.transform(y_train)

X_valid= scalerX.transform(X_valid)
#y_valid_scale = scalery.transform(y_valid)

X_test= scalerX.transform(X_test)
#y_test_scale = scalery.transform(y_test)

In [ ]:
#텐서화
#X_train = xgb.DMatrix(X_train_scale)
#y_train = xgb.DMatrix(y_train.values)

#X_valid = xgb.DMatrix(X_valid_scale)
#y_valid = xgb.DMatrix(y_valid.values)

#X_test = xgb.DMatrix(X_test_scale)
#y_test = xgb.DMatrix(y_test.values)

In [ ]:
os.makedirs('./weights', exist_ok=True)

In [ ]:

# this is parameter tuning result
"""kfol=KFold(n_splits=3,random_state=2022,shuffle=True)
xgb1 = xgb.XGBRegressor( tree_method='gpu_hist')
"""

"""
parameters = {
              'objective':['reg:squarederror'],
              'learning_rate': [0.01,1e-5], 
              'max_depth': [3,5,7,9],
              'min_child_weight': [1,3,4,5],
              'subsample': [0.6,0.7,0.8,1],
              'colsample_bytree':[i/10.0 for i in range(6,10)],
              'n_estimators': [100,200,500,750,1000]
              #'gamma':[i/10.0 for i in range(0,5)],
              #'reg_lambda':[1,0.9,0.8,0.7,0],
              #'reg_alpha':[0, 0.001, 0.005, 0.01, 0.05, 0.1, 1],
              }
"""
# another parameter tuning
"""
parameters={
    'n_estimators': [100,200,500,750,1000],
    'max_depth': [3,5,7,9],
    'min_child_weight': [1,3,5],
    'gamma':[i/10.0 for i in range(0,5)],
    'subsample':[i/10.0 for i in range(6,10)],
    'colsample_bytree':[i/10.0 for i in range(6,10)],
    'reg_alpha':[0, 0.001, 0.005, 0.01, 0.05, 0.1, 1],
    'learning_rate': [0.01, 0.02, 0.05, 0.1]
}
"""

"""
parameters= {'colsample_bytree': [0.9], 
              'learning_rate': [0.01], 
              'max_depth': [9], 
              'min_child_weight': [3], 
              'n_estimators': [1000], 
              'objective': ['reg:squarederror'], 
              'subsample': [0.7],
              'gamma':[i/10.0 for i in range(0,5)],
              'reg_lambda':[1,0.9,0.8,0.7,0],
              'reg_alpha':[0, 0.001, 0.005, 0.01, 0.05, 0.1, 1],
              }
"""

In [ ]:
# using Grid Search, we can get params tuning
"""
xgb_grid = GridSearchCV(xgb1,
                        parameters,
                        cv = kfol,
                        n_jobs = 3,
                        verbose=0
                        )                     
"""

In [ ]:
# do the grid search and print the result
"""
xgb_grid.fit(X_train,y_train,early_stopping_rounds=20,eval_metric='rmse',eval_set=[(X_valid,y_valid)],verbose=0)


print(xgb_grid.best_score_)
print(xgb_grid.best_params_)"""

In [ ]:

#print(xgb_grid.best_score_)
#print(xgb_grid.best_params_)

### 5/5 
# best parameter
bestparm={'colsample_bytree': 0.9,
          'gamma': 0.3,
          'learning_rate': 0.01,
          'max_depth': 9,
          'min_child_weight': 3,
          'n_estimators': 1000,
          'objective': 'reg:squarederror',
          'reg_alpha': 1,
          'reg_lambda': 0.9,
          'subsample': 0.7}

In [ ]:
"""

best_params= {'colsample_bytree': 0.9, 
              'learning_rate': 0.01, 
              'max_depth': 9, 
              'min_child_weight': 3, 
              'n_estimators': 1000, 
              'objective': 'reg:squarederror', 
              'subsample': 0.7}
"""

In [ ]:
# change the data to fit the model format
d_train=xgb.DMatrix(data=X_train,label=y_train)
d_valid=xgb.DMatrix(data=X_valid,label=y_valid)
wlist=[(d_train,'train'),(d_valid,'eval')]
#best_params=xgb_grid.best_params_

In [ ]:
# set the baseline_train the model
model = xgb.train(params=bestparm,
                  dtrain=d_train,
                  #eval_metric='mae',
                  evals=wlist,
                  num_boost_round=5000
                  #tree_method='gpu_hist'
                )

In [ ]:
# see the test set
X_test.head()

In [ ]:
# fit to the format
d_test=xgb.DMatrix(data=X_test,label=y_test)

In [ ]:
# predict result
preds= model.predict(d_test)

In [ ]:
# to extract information by metric
def testing(pred,y):
    sums1=[]
    sums2=[]
    for i in range(len(preds)):
        sqr=(float(pred[i])-float(y.loc[i]))**2
        abs1=abs(float(pred[i])-float(y.loc[i]))
        sums1.append(sqr)
        sums2.append(abs1)
        
        
    rmse=(sum(sums1)/len(preds))**0.5
    mae= sum(sums2)/len(preds)
    return rmse,mae

In [ ]:
# print the test result
print('rmse =', testing(preds,y_test)[0])
print('mae =', testing(preds,y_test)[1])